In [33]:
import pandas as pd
import numpy as np
import os

In [21]:
def read_bed(file):
    data = pd.read_csv(file, sep='\t', header=None)
    data.columns = ['chr', 'start', 'end', 'additional_data', 'value', 'orientation', 'start2', 'end2'][:data.columns.shape[0]]
    return data

In [22]:
file = read_bed('sorted_data/mm9_Histone_H3K79me3.sorted.bed')
file

,chr,start,end,additional_data,value,orientation,start2,end2
0,chr1,3129251,3129334,NaN,208,+,3129251,3129334
1,chr1,12647561,12647848,NaN,292,+,12647561,12647848
2,chr1,14633906,14633993,NaN,203,+,14633906,14633993
3,chr1,16333014,16333177,NaN,221,+,16333014,16333177
4,chr1,16333177,16333178,NaN,154,+,16333177,16333178
...,...,...,...,...,...,...,...,...
592,chrX,166527237,166527500,NaN,1820,+,166527237,166527500
593,chrX,166650059,166650280,NaN,2004,+,166650059,166650280
594,chrY,2497956,2498032,NaN,203,+,2497956,2498032
595,chrY,2652210,2652212,NaN,1183,+,2652210,2652212


In [36]:
def make_data_by_chroms(file):
    return {
        chr: file[file.chr == chr].sort_values(by='start')
        for chr in file.chr.value_counts().index
    }

In [37]:
def check_intersections(data_by_chroms):
    all_errors = 0
    all_errors_by_chroms = []
    for chr, data in data_by_chroms.items():
        # print(f'chrome: {chr}', end=' ... ')
        errors = (np.array(data.start[1:]) < np.array(data.end[:-1])).sum()
        all_errors += errors
        all_errors_by_chroms.append(errors)
        # if errors == 0:
            # print('ok')
        # else:
            # print(f'fail: {errors} out of {data.shape[0] - 1}' )
    return all_errors, all_errors_by_chroms

In [41]:
data_dir = 'sorted_data'

problem_files = []
for file_name in os.listdir(data_dir):
    check_result = check_intersections(
        make_data_by_chroms(
            read_bed(
                os.path.join(data_dir, file_name)
            )
        )
    )
    if check_result[0] != 0:
        problem_files.append((file_name, check_result))
print('Files processed')
print('Problem files:')
for file_name, check_result in problem_files:
    print(f'{file_name}: {check_result[0]}, {check_result[1]}')

Files processed
Problem files:
